In [609]:
import serial
import time
import csv
import sys
import random
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collections import deque
import threading
import pandas as pd

In [610]:
start_time = []
start_time.append(time.time())
duration = 10

time_val_list = []
load_val_list = []
deflection_val_list = []

sub_deflection_list = []

final_time_list = []
final_load_list = []
final_deflection_list = []

synthetic_time_list = []

increment_val = []

In [611]:
def load_deflection_val():
    load = np.random.normal(0, 100)
    deflection = np.random.normal(0, 10)
    
    elapsed_time = time.time()
    time.sleep(0.005)

    deflection_val_list.append(deflection)
    load_val_list.append(load)
    time_val_list.append(elapsed_time)
    print('start time:', start_time[0])
    print('current time:', time.time())
    print('elapsed time:',elapsed_time)

In [612]:
def synthetic_sensor_data():
    refinement = 10
    if len(sub_deflection_list) == refinement:
        sub_deflection_list.clear()
        load_deflection_val()
    else:
        if len(sub_deflection_list) < refinement:
            if len(deflection_val_list) == 0:
                i = 1
                while True:
                    load_deflection_val()
                    if i%2 == 0:
                        break
                    i = i+1
            else:
                x2 = deflection_val_list[-1]
                l2 = load_val_list[-1]
                t2 = time_val_list[-1]

                x1 = deflection_val_list[-2]
                l1 = load_val_list[-2]
                t1 = time_val_list[-2]

                for i in range(0, refinement + 1, 1):
                    sub_deflection_val = x1 + ((x2 - x1) / refinement) * (len(sub_deflection_list)-1)
                    sub_deflection_list.append(sub_deflection_val)
                    final_deflection_list.append(sub_deflection_val)

                    sub_load_val = l1 + ((l2 - l1) / refinement) * (len(sub_deflection_list))
                    final_load_list.append(round((sub_load_val), 2))

                    sub_time_val = t1 + ((t2 - t1) / refinement) * (len(sub_deflection_list)-1)
                    final_time_list.append(sub_time_val)
                    return

synthetic_sensor_data()
print(time_val_list)
print(final_time_list)
print(len(final_time_list))
print(len(time_val_list))

start time: 1737409504.2688613
current time: 1737409504.2819388
elapsed time: 1737409504.2819388
start time: 1737409504.2688613
current time: 1737409504.282937
elapsed time: 1737409504.2819388
[1737409504.2819388, 1737409504.2819388]
[]
0
2


In [630]:
for i in range(0,2,1):
    print(time.time())

1737409655.2288306
1737409655.2288306
